In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns



In [4]:
# Convert Hour - Minute Column from ""newsdata/Master Thesis - News Overview - Election News.csv"" the proper format

# Load the news event CSV
news_df = pd.read_csv("/Users/juliuswalkenhorst/masterthesis/newsdata/Master Thesis - News Overview - Election News.csv")

# Rename for convenience
news_df = news_df.rename(columns={"Hour - Minute (UTC)": "raw_event_time"})

# Clean up the a.m. / p.m. to AM / PM
news_df["raw_event_time"] = (
    news_df["raw_event_time"]
    .str.replace("a.m.", "AM", regex=False)
    .str.replace("p.m.", "PM", regex=False)
)

# Convert to datetime
news_df["event_timestamp"] = pd.to_datetime(
    news_df["raw_event_time"], 
    format="%I:%M %p on %B %d, %Y"
)

In [5]:
print(news_df.head())

                                               Event                 Day  \
0                            Super Tuesday Primaries       March 5, 2024   
1  Donald Trump found guilty in New York trial (3...        May 30, 2024   
2  Trump survives sniper assassination attempt at...       July 13, 2024   
3  Republican National Convention — Trump officia...  July 15 - 18, 2024   
4   President Biden Withdraws from Presidential Race       July 21, 2024   

                                        Link to News  Sentiment  \
0  https://vote.org/election-calendar/ (Super Tue...        NaN   
1  https://www.nytimes.com/live/2024/05/30/nyregi...        NaN   
2  https://abcnews.go.com/US/timeline-donald-trum...        NaN   
3  https://www.cbsnews.com/news/rnc-roll-call-of-...        NaN   
4  https://www.nytimes.com/live/2024/07/21/us/bid...        NaN   

  Hour - Minute (EDT)            raw_event_time  \
0      11:00 p.m. EST  4:00 AM on March 6, 2024   
1           4:55 p.m.   8:55 PM on May

In [8]:
# Clean up column names (remove leading/trailing spaces)
news_df.columns = news_df.columns.str.strip()


In [9]:
# Create event_timestamp in electionmaster.csv
election_df = pd.read_csv("/Users/juliuswalkenhorst/masterthesis/csvfileselection/electionmaster.csv")

# Extract label (Dune query ID) from the Dune URL
news_df["label"] = news_df["Dune Script"].str.extract(r'(\d+)$')

# Make sure the label in both DataFrames is the same type (string)
election_df["label"] = election_df["label"].astype(str)
news_df["label"] = news_df["label"].astype(str)

# Merge event_timestamp into the election_df using label
merged_df = election_df.merge(
    news_df[["label", "event_timestamp"]],
    on="label",
    how="left"
)

In [13]:
print(merged_df.columns)

Index(['buyer', 'buyerPaid', 'evt_tx_hash', 'sharesBought', 'sharesType',
       'takerAssetID', 'time', 'label', 'probabilities', 'event_timestamp'],
      dtype='object')


In [22]:
# Print the first 5 rows of merged_df time and event_timestamp
print(merged_df[["time", "event_timestamp"]].head())



                          time     event_timestamp
0  2024-07-22 15:29:21.000 UTC 2024-07-21 15:30:00
1  2024-07-22 15:28:43.000 UTC 2024-07-21 15:30:00
2  2024-07-22 15:28:29.000 UTC 2024-07-21 15:30:00
3  2024-07-22 15:26:37.000 UTC 2024-07-21 15:30:00
4  2024-07-22 15:26:37.000 UTC 2024-07-21 15:30:00


In [25]:
# Strip " UTC" if still present
merged_df["time"] = merged_df["time"].str.replace(" UTC", "", regex=False)

# Let pandas infer each format individually
merged_df["time"] = pd.to_datetime(merged_df["time"], format='mixed', errors='coerce')

# Just in case, ensure event_timestamp is parsed too
merged_df["event_timestamp"] = pd.to_datetime(merged_df["event_timestamp"], errors='coerce')

# Compute the time difference in minutes
merged_df["minutes_from_event"] = (
    (merged_df["time"] - merged_df["event_timestamp"]).dt.total_seconds() / 60
)


In [27]:
print(merged_df["minutes_from_event"].tail(20))

296118   -1432.966667
296119   -1432.966667
296120   -1432.966667
296121   -1432.966667
296122   -1432.966667
296123   -1432.966667
296124   -1432.966667
296125   -1432.966667
296126   -1435.133333
296127   -1435.133333
296128   -1435.300000
296129   -1435.400000
296130   -1435.400000
296131   -1435.500000
296132   -1435.500000
296133   -1435.600000
296134   -1435.633333
296135   -1435.633333
296136   -1440.000000
296137   -1440.000000
Name: minutes_from_event, dtype: float64


In [28]:
print(merged_df["probabilities"].head(20))

0     0.650000
1     0.650000
2     0.290000
3     0.290000
4     0.710000
5     0.284000
6     0.283000
7     0.292108
8     0.284000
9     0.283000
10    0.717000
11    0.284000
12    0.283000
13    0.283000
14    0.290000
15    0.284000
16    0.714136
17    0.290000
18    0.710000
19    0.360000
Name: probabilities, dtype: float64


In [29]:
# Count how many rows have NaT in minutes_from_event
num_nat_rows = merged_df["minutes_from_event"].isna().sum()
print(f"Number of rows with NaT in minutes_from_event: {num_nat_rows}")


Number of rows with NaT in minutes_from_event: 0


In [30]:
print(merged_df.columns)

Index(['buyer', 'buyerPaid', 'evt_tx_hash', 'sharesBought', 'sharesType',
       'takerAssetID', 'time', 'label', 'probabilities', 'event_timestamp',
       'minutes_from_event'],
      dtype='object')


In [31]:
print(merged_df["sharesType"].unique())


['Trump Wins - YES' 'Kamala Harris Wins - YES' 'Kamala Harris Wins - NO'
 'Trump Wins - NO' 'Biden Wins - YES' 'Biden Wins - NO']


In [33]:
yes_df = merged_df[merged_df["sharesType"].str.endswith("YES")].copy()
yes_df["candidate"] = yes_df["sharesType"].str.replace(" Wins - YES", "")

In [34]:
resampled_dfs = []

for (label, candidate), group in yes_df.groupby(["label", "candidate"]):
    df = group.set_index("time").sort_index()
    df_resampled = df.resample("1min").mean(numeric_only=True).reset_index()
    df_resampled["label"] = label
    df_resampled["candidate"] = candidate
    resampled_dfs.append(df_resampled)

# Combine all resampled results
resampled_yes_df = pd.concat(resampled_dfs, ignore_index=True)


In [35]:
# Save the resampled candidate-level probabilities
resampled_yes_df.to_csv("resampled_yes_df.csv", index=False)

# Save the full merged dataset with event timestamps and minutes_from_event
merged_df.to_csv("electionmastermerged.csv", index=False)
